In [1]:
import pandas as pd 
import numpy as np 
import math
import pandas as pd
import sys
from apyori import apriori
from tqdm import tqdm
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
class Error(Exception):
    def __str__(self):
        return "허용되지 않는 행동입니다."

In [3]:
print(sys.path[0])

/home/shinjk1156/Project_code/Retail_Project/Code


### Setting Grid World

In [4]:
df = pd.read_excel('/home/shinjk1156/Project_code/Retail_Project/Data/RFID_IPS_데이터추출_20200904.xlsx', sheet_name= None)

In [5]:
grid_dic = {"청과" : [(0,1),(0,2),(0,3),(0,4)], "곡물" : [(1,1),(1,2),(1,3),(1,4)], "입구" : [(0,0)], 
            "채소" : [(0,5),(0,6),(0,7),(1,5),(1,6),(1,7)], "닭고기" : [(0,8)], "수산" : [(0,9), (1,8), (1,9)], "축산" : [(0,10), (1,10)], 
            "델리카" : [(0,11), (1,11)], "계산대" : [(1,0),(2,0),(3,0),(4,0)], "청소욕실" : [(2,5),(2,6),(3,5),(3,6)],
            "주방용품" : [(2,7),(3,7)], "언더웨어" : [(4,5), (4,6)], "H&B" : [(2,1),(2,2),(2,3),(2,4), (3,1),(3,2),(3,3),(3,4), (4,1),(4,2),(4,3),(4,4)],
            "건강,차" : [(4,7)], "조미식품" : [(2,8), (2,9)], "인스턴트" : [(3,8), (3,9)], "과자" : [(4,8), (4,9)], 
            "냉장냉동" : [(2,10), (2,11)], "데일리" : [(3,10), (3,11)], "음료,주류" : [(4,10),(4,11)],
            "스포츠" : [(5,1)], "문구" : [(5,2)], "완구" : [(5,3)], "자동차" : [(5,4)], "애완원예용품" : [(5,5)], "인테리어" : [(5,6)],
            "수예" : [(5,7)], "세제" : [(5,8)], "위생용품" : [(5,9)], "금지구역" : [(5,10),(5,11),(5,0)]}

In [6]:
class Agent:
    def __init__(self, initial_position):
        self.action_space = np.array([(0,0), (-1,0), (1,0), (0,-1), (0,1)])
        self.action_pr = np.array([0.20,0.20,0.20,0.20,0.20])
        self.pos = np.array(initial_position)

    def set_pos(self, position):
        self.pos = position
        return self.pos

    def get_pos(self):
        return self.pos

In [7]:
class Env:
    # action space = [(0,0)), (-1,0)), (1,0)), (0,-1)), (0,1)]
    def __init__(self, grid_dic):
        self.grid_dic = grid_dic
        self.grid_world = self.make_grid_world()

    def make_grid_world(self):
        grid_world = pd.DataFrame(np.zeros((6,12)))
        for key in self.grid_dic.keys():
            for x,y in self.grid_dic[key]:
                grid_world.iloc[x,y] = key
        return grid_world

    def move(self, agent, action):
        terminal = False
        reward = 0
        #현재 좌표
        current_pos = agent.pos
        #현재 좌표에 대응되는 현재 매대
        cur_key = self.grid_world.iloc[current_pos[0], current_pos[1]]
        #이동된 좌표
        try: next_pos = agent.pos + agent.action_space[action] #아직은 전이확률을 고려하지 않음
        except: raise Error()
        #이동된 좌표에 대응되는 다음 매대
        try: next_key = self.grid_world.iloc[next_pos[0], next_pos[1]]
        except: next_key = "금지구역"

        #매대간 이동이 존재하는대도 만약 cur_key와 next_key가 같을 시,(이동이 발생했는대도 그리드월드 설계문제로 인해 매대 이동이 발생하지 못함)
        #이 때 이동좌표를 변경시켜줌
        if action != 0:
            if cur_key == next_key:
                temp = np.array(self.grid_dic[next_key])
                if action == 1: next_pos[0] = min(temp.T[0]) - 1
                elif action == 2: next_pos[0] = max(temp.T[0]) + 1
                elif action == 3: next_pos[1] = min(temp.T[1]) - 1
                elif action == 4: next_pos[1] = max(temp.T[1]) + 1
                else: raise Error()
        #현재 좌표가 최종목적지인지 확인
        if self.grid_world.iloc[current_pos[0], current_pos[1]] == "계산대":
            terminal = True
            show_next_pos = agent.set_pos(agent.pos)
            #reward = 0

        #이동된 좌표가 이동 불가능한 지점인지 확인
        elif next_key == "금지구역" or tuple(next_pos) in self.grid_dic["금지구역"] or next_pos[0] < 0 or next_pos[1] < 0:
            terminal = True
            show_next_pos = agent.set_pos(agent.pos)
            #reward = 0

        #이동가능한 동선이면 '이동':
        else:
            show_next_pos = agent.set_pos(next_pos)
        
        return show_next_pos, reward, terminal



In [8]:
agent = Agent((1,1)) 
env = Env(grid_dic)
print(env.move(agent, 4))
agent.get_pos()

(array([1, 5]), 0, False)


array([1, 5])

In [9]:
env.grid_world

,0,1,2,3,4,5,6,7,8,9,10,11
0,입구,청과,청과,청과,청과,채소,채소,채소,닭고기,수산,축산,델리카
1,계산대,곡물,곡물,곡물,곡물,채소,채소,채소,수산,수산,축산,델리카
2,계산대,H&B,H&B,H&B,H&B,청소욕실,청소욕실,주방용품,조미식품,조미식품,냉장냉동,냉장냉동
3,계산대,H&B,H&B,H&B,H&B,청소욕실,청소욕실,주방용품,인스턴트,인스턴트,데일리,데일리
4,계산대,H&B,H&B,H&B,H&B,언더웨어,언더웨어,"건강,차",과자,과자,"음료,주류","음료,주류"
5,금지구역,스포츠,문구,완구,자동차,애완원예용품,인테리어,수예,세제,위생용품,금지구역,금지구역


### Association Rule Mining

In [28]:
pos_detail = df['POS_DETAIL']
pog_cate = df["POG-카테고리"]
pog = df["POG"]
info_item = df["상품정보"]
info_cate = df['상품-카테고리']

In [29]:
pog_df = pd.merge(pog_cate, pog, on = "POG 그룹 ID", how = "left")
pog_df.rename(columns = {"DIVISION" : "DIV_CD", "DEPT" : "DEPT_CD", "POG 이름_x" : "POG 이름", "POG 이름_y":"POG 그룹 이름"}, inplace = True)
pog_df.drop(index = 0, inplace = True)
pog_df.drop(columns = ["사용 유무", "POG 그룹 ID", "POG ID","POG 이름"], inplace = True)
pog_df.drop_duplicates(inplace = True)

In [30]:
#info_item.drop(index = 0, inplace = True)
info_item.rename(columns = {"DIVISION CODE":"DIV_CD", "DEPT CODE" : "DEPT_CD", "SECTION CODE" : "SECTION_CD", "CLASS CODE" : "CLASS_CD"}, inplace= True)
info_df = pd.merge(info_item, info_cate, on = ["DIV_CD", "DEPT_CD", "SECTION_CD", "CLASS_CD"], how = "left")
info_df.drop(columns= ["바코드번호", "판매 가격"], inplace= True)
pog_in_df = pd.merge(info_df, pog_df, on = ["DIV_CD", "DEPT_CD"], how = "left")
item_df = pd.merge(pos_detail.loc[:,["쇼핑 ID", "자체상품코드"]], pog_in_df, on = "자체상품코드", how = "left")
item_df.rename(columns= {"DIVISION CODE" : "DIVISION", "DEPT CODE" : "DEPT"}, inplace = True)
item_df.drop(index= 0, inplace = True)
item_df.drop_duplicates(inplace = True)
item_df.reset_index(drop= True, inplace = True)

In [31]:
item_df

,쇼핑 ID,자체상품코드,상품명,DIV_CD,DEPT_CD,SECTION_CD,CLASS_CD,DIV_NAME,DEPT_NAME,SECTION_NAME,CLASS_NAME,POG 그룹 이름
0,1,1355218000,●동원 참치세트 16호,27,185,1617,11400,인스턴트,선물세트,통조림선물세트,참치통조림혼합선물세트,NaN
1,1,0002960000,피홍합,20,154,1452,13770,생선,해물연체류,각패류,홍합,NaN
2,1,1289860000,다이소2000원균일가제품,35,782,7238,67347,주방용품,주방시즌/균일가,주방균일가,주방용품균일가,NaN
3,1,1260181000,와이즐렉 네임펜흑3입,39,256,1849,12020,문구,필기도구,펜,유성매직,문구
4,1,1248045000,와이즐렉스테플러10호세트,39,258,1860,12072,문구,일반사무용품,OA용품,스테플러,문구
...,...,...,...,...,...,...,...,...,...,...,...,...
356082,26043,1480971000,풀)국산 더단단한 부침두부,19,146,7250,67537,채소,두부,풀무원두부,풀무원포장두부,채소2
356083,26043,1494164000,아침의 계란(왕란/15입),72,805,7378,67992,닭고기,계란,일반란,소포장란,닭고기
356084,26045,0008750000,친환경 쌈채소,19,149,1391,10624,채소,친환경채소,친환경쌈채류,친환경쌈채류,채소2
356085,26045,0030732000,바지락살,20,154,1453,10751,생선,해물연체류,살패류,바지락살,NaN


In [32]:
#DIV_NAME들을 구분하여 GW카테고리에 맞춰 새로운 칼럼생성
gw_list = []
for i in range(len(item_df)):
    name = item_df.loc[i, "DIV_NAME"]
    if name == "과일":
        gw_list.append("청과")
    elif name in ["생선","건해산물"]:
        gw_list.append("수산")
    elif name in ["돼지고기" ,"소고기"]:
        gw_list.append("축산")
    elif name == "조리식품":
        gw_list.append("델리카")
    elif name in ["H&B", "섬유잡화", "장신잡화", "남성의류편집", "피혁잡화", "신발", "아동유아편집"]:
        gw_list.append("H&B")
    elif name in ["음료", "주류"]:
        gw_list.append("음료,주류")
    elif name in ["일반스포츠", "레져스포츠", "스포츠NB"]:
        gw_list.append("스포츠")
    else:
        gw_list.append(name)
item_df["GW_Cate"] = gw_list
item_df.drop(columns = ["POG 그룹 이름", "DEPT_NAME","SECTION_NAME", "CLASS_NAME"], inplace= True)
item_df.drop_duplicates(inplace = True)
item_df.dropna(inplace = True)
item_df.reset_index(drop = True, inplace= True)

In [33]:
shid_list = item_df["쇼핑 ID"].unique()
transaction_list = []
for s_id in tqdm(shid_list, desc = "Create Transaction_list"):
    transaction_list.append(list(item_df[item_df["쇼핑 ID"] == s_id].GW_Cate.unique()))


100%|██████████| 13864/13864 [02:14<00:00, 102.79it/s]


In [34]:
t_encoder = TransactionEncoder()
t_array = t_encoder.fit(transaction_list).transform(transaction_list)
t_df = pd.DataFrame(t_array, columns= t_encoder.columns_)

In [53]:
fre_itemset = apriori(t_df, min_support= 0.05, use_colnames= True)
fre_itemset.sort_values(by = "support", ascending = False, inplace = True)
fre_itemset.reset_index(drop = True, inplace= True)
fre_itemset

,support,itemsets
0,0.707372,(채소)
1,0.569749,(데일리)
2,0.462709,(청과)
3,0.452178,(수산)
4,0.419432,"(데일리, 채소)"
...,...,...
693,0.050202,"(인스턴트, 수산, 주방용품, 채소)"
694,0.050130,"(인스턴트, 데일리, 델리카, 채소, 청과)"
695,0.050058,"(청소욕실, 데일리, 냉장냉동)"
696,0.050058,"(인스턴트, H&B, 냉장냉동)"


In [61]:
import csv

In [63]:
with open("..rules.csv", 'w', newline = '') as file:
    writer = csv.writer(file)
    writer.writerows(item_list)

In [59]:
min_num_items = 5
num_collect = 10
i = 0
count = 0
item_list=[]
while(True):
    if count == num_collect:
        break
    if len(fre_itemset.itemsets[i]) >= min_num_items:
        count += 1
        item_list.append(list(fre_itemset.itemsets[i]))
    i+=1

return item_list


In [37]:
association_rules(fre_itemset, metric="confidence", min_threshold=0.3) 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(H&B),(과자),0.196336,0.418782,0.098745,0.502939,1.200955,0.016523,1.169308
1,(H&B),(냉장냉동),0.196336,0.383078,0.086267,0.439383,1.146979,0.011055,1.100433
2,(H&B),(닭고기),0.196336,0.261829,0.059290,0.301984,1.153362,0.007884,1.057527
3,(H&B),(데일리),0.196336,0.569749,0.126010,0.641807,1.126474,0.014148,1.201173
4,(H&B),(델리카),0.196336,0.333309,0.070326,0.358193,1.074655,0.004885,1.038771
...,...,...,...,...,...,...,...,...,...
3499,"(조미식품, 데일리, 채소)","(인스턴트, 청과)",0.147144,0.192297,0.055251,0.375490,1.952662,0.026956,1.293340
3500,"(조미식품, 청과, 데일리)","(인스턴트, 채소)",0.101125,0.291474,0.055251,0.546362,1.874478,0.025776,1.561876
3501,"(조미식품, 청과, 채소)","(인스턴트, 데일리)",0.128318,0.264786,0.055251,0.430579,1.626136,0.021274,1.291160
3502,"(인스턴트, 조미식품)","(청과, 데일리, 채소)",0.162074,0.240118,0.055251,0.340899,1.419713,0.016334,1.152906


### Sequence Pattern Mining

In [ ]:
df = pd.read_excel('/home/shinjk1156/Project_code/Retail_Project/Data/RFID_IPS_데이터추출_20200904.xlsx', sheet_name= None)
line = df['라인정보']
repeater = df["리피터 마스터"]
pog = df['POG']
pog_cate = df['POG-카테고리']
shoping = df['가공된 쇼핑 데이터']
#pog.drop(index= 0, inplace = True)
#repeater.drop(index= 0, inplace = True)
pog["POG 그룹 ID"] = pog["POG 그룹 ID"].map(str)
pog_cate["POG 그룹 ID"] = pog_cate["POG 그룹 ID"].map(str)
moving1 = pd.read_csv("실시간저장데이터_20110208_20110220.txt", sep = "\t", header = None)
moving2 = pd.read_csv("실시간저장데이터_20110117_20110207.txt", sep = "\t", header = None)
moving = moving1.append(moving2)
moving.columns = ["SINK ID", "태그 ID","TAG 강도", "리피터 ID", "저장시간"]

repeater["리피터 ID"] = repeater["리피터 ID"].astype(str)
moving["리피터 ID"] = moving["리피터 ID"].astype(str)

mer_df = pd.merge(repeater, pog, on = "POG 그룹 ID", how = "left")

to_shop = pd.DataFrame()
for i in tqdm(range(1,len(shoping))):
    sample = moving[(moving.저장시간 > shoping.loc[i,"쇼핑 시작시간"]) & (moving.저장시간 < shoping.loc[i,"쇼핑 종료 시간"]) & (moving["태그 ID"] ==shoping.loc[i, "태그 ID"])].copy()
    if len(sample) == 0:
        pass
    else:
        sample.sort_values(by = "저장시간", inplace = True)
        sample = pd.merge(sample, repeater, on = "리피터 ID", how = "left")
        sample["쇼핑 ID"] = [shoping.loc[i, "쇼핑 ID"]]*len(sample)
        to_shop = to_shop.append(sample)
#to_shop.to_csv("동선_전처리_0222_0220.csv", index = False)


In [ ]:
fdf = to_shop.copy()
fdf.drop(columns = ["사용 강도", "상품 여부","가중치","리포트에 보여줄지에 대한 여부","스타트 여부","사용여부","POS 여부", "TAG 강도"], inplace = True)
fdf.dropna(inplace = True)

In [ ]:
info_item = df["상품정보"]
info_cate = df["상품-카테고리"]
info_item.rename(columns = {"DIVISION CODE":"DIV_CD", "DEPT CODE" : "DEPT_CD", "SECTION CODE" : "SECTION_CD", "CLASS CODE" : "CLASS_CD"}, inplace= True)
info_df = pd.merge(info_item, info_cate, on = ["DIV_CD", "DEPT_CD", "SECTION_CD", "CLASS_CD"], how = "left")
info_df.drop(columns= ["바코드번호", "판매 가격"],index = 0 ,inplace= True)
info_df

In [ ]:
pog_df = pd.merge(pog_cate, pog, on="POG 그룹 ID", how = "left")
pog_df.rename(columns = {"DIVISION" : "DIV_CD", "DEPT" : "DEPT_CD", "POG 이름_x" : "POG 이름", "POG 이름_y":"POG 그룹 이름"}, inplace = True)
pog_df.drop(index = 0, inplace = True)
pog_df.drop(columns = ["사용 유무", "POG ID","POG 이름","DEPT_CD"], inplace = True)
pog_df.drop_duplicates(inplace = True)
pog_df

In [ ]:
pog_in_df = pd.merge( pog_df, info_df.loc[:,["DIV_CD", "DIV_NAME"]], on = ["DIV_CD"], how = "left")

In [ ]:
pog_in_df.drop(columns = ["POG 그룹 이름"], inplace = True)
pog_in_df.drop_duplicates(inplace = True)
pog_in_df.reset_index(drop = True, inplace = True)

In [ ]:
#DIV_NAME들을 구분하여 GW카테고리에 맞춰 새로운 칼럼생성
gw_list = []
for i in range(len(pog_in_df)):
    name = pog_in_df.loc[i, "DIV_NAME"]
    if name == "과일":
        gw_list.append("청과")
    elif name in ["생선","건해산물"]:
        gw_list.append("수산")
    elif name in ["돼지고기" ,"소고기"]:
        gw_list.append("축산")
    elif name == "조리식품":
        gw_list.append("델리카")
    elif name in ["H&B", "섬유잡화", "장신잡화", "남성의류편집", "피혁잡화", "신발", "이지캐주얼" , "아동유아편집","여성의류편집","여성의류NB","남성의류NB","아동유아NB"]:
        gw_list.append("H&B")
    elif name in ["음료", "주류"]:
        gw_list.append("음료,주류")
    elif name in ["일반스포츠", "레져스포츠", "스포츠NB"]:
        gw_list.append("스포츠")
    else:
        gw_list.append(name)

In [ ]:
pog_in_df["GW_Cate"] = gw_list
pog_in_df.drop(columns = ["DIV_CD", "DIV_NAME"],inplace = True)
pog_in_df.drop_duplicates(inplace = True)

In [ ]:
pog_id = []
gw_ca = []
for i in range(17):
    if i == 0:
        pog_id.append(0)
        gw_ca.append("입구")
    else:
        pog_id.append(i)
        gw_ca.append("계산대")
io_df = pd.DataFrame()
io_df["POG 그룹 ID"] = pog_id
io_df["GW_Cate"] = gw_ca

In [ ]:
pog_in_df = pd.concat([pog_in_df, io_df])
pog_in_df["POG 그룹 ID"] = pog_in_df["POG 그룹 ID"].astype(int)
fdf["POG 그룹 ID"] = fdf["POG 그룹 ID"].astype(int)
final = pd.merge(fdf, pog_in_df, on= "POG 그룹 ID", how = "left")


In [ ]:
final.to_csv("동선정보_매핑완료_0122_0220.csv", index = False)